In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 

from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns


sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

########################################

def SIR_model(SIR,beta,gamma,N0):
    '''Simple SIR model
        S: susceptible population
        I: infected population
        R: recovered population
        beta: infection rate
        gamma: recovery rate
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
     Parameters:
        ----------
        SIR : numpy.ndarray
        beta: float
        gamma: float
    '''
    
    S,I,R = SIR
    dS_dt=-beta*S*I/N0
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return(dS_dt,dI_dt,dR_dt)



if __name__ == '__main__':
    
    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()


In [2]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())

df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')

fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## SIR Parameters
        '''),
    
    dcc.Markdown('''
    ## Initial period(in Days)
    '''),
    dcc.Input(
            id="t_initial", type="number", placeholder="number",
            value=21,min=0, max=100, 
            step=1,debounce=True
    ),
    
     dcc.Markdown('''
    ## Measures introduced period(in Days)
    '''),
    dcc.Input(
            id="t_intro_measures", type="number", placeholder="number",
            value=14,min=0, max=100,
            step=1,debounce=True
    ),
    
     dcc.Markdown('''
    ## Holding period(in Days)
    '''),
    dcc.Input(
            id="t_hold", type="number", placeholder="number",
            value=21,min=0, max=100,
            step=1,debounce=True
    ),
    
     dcc.Markdown('''
    ## Relaxation period(in Days)
    '''),
    dcc.Input(
            id="t_relax", type="number", placeholder="number",
            value=21,min=0, max=1000,
            step=1,debounce=True
    ),
    
     dcc.Markdown('''
    ## Maximum infection rate
    '''),
     dcc.Input(
             id="beta_max", type="number", placeholder="number",
             value=0.4,min=0, max=100,
             step=0.1,debounce=True
             
    ),
    
     dcc.Markdown('''
    ## Minimum infection rate
    '''),
     dcc.Input(
             id="beta_min", type="number", placeholder="number",
             value=0.11,min=0, max=100,
             step=0.01,debounce=True
    ),
    
     dcc.Markdown('''
    ## Recovery rate
    '''),
     dcc.Input(
             id="gamma", type="number", placeholder="number",
             value=0.1,min=0, max=100,
             debounce=True
    ),
    
    dcc.RadioItems(
        id="change_scale",
    options=[
        {'label': 'linear', 'value': 'linear'},
        {'label': 'log', 'value': 'log'},
       
    ],
    value='log',
    labelStyle={'display': 'inline-block'}
),  
    
    dcc.Graph(figure=fig, id='main_window_slope')
])

        

X:\CVT\CVT\Computer\data_science\eds_covid-19\notebooks


In [3]:
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('t_initial', component_property='value'),
    Input('t_intro_measures',component_property= 'value'),
    Input('t_hold',component_property= 'value'),
    Input('t_relax',component_property= 'value'),
    Input('beta_max',component_property= 'value'),
    Input('beta_min', component_property='value'),
    Input('gamma', component_property='value'),
    Input('change_scale', component_property='value')])

def update_figure(country_list,t_init,t_intro,t_hold,t_relax,bmax,bmin,gamma,change_scale):
    
    traces =[]
    print(gamma)
    print(change_scale)
    
    t_phases=np.array([t_init,t_intro,t_hold,t_relax]).cumsum()
    
    if 'log' in change_scale:
        my_yaxis={'title':'Confirmed infected people (Source: John Hopkins,log-scale)',
                  'type':"log",
                  'tickfont':dict(size=14,color="#7f7f7f"),
                                  }
    else:
        my_yaxis={'title':'Confirmed infected people (Source: John Hopkins,linear-scale)',
                  'type':"linear",
                  'tickfont':dict(size=14,color="#7f7f7f"),
                                  }
        print(my_yaxis)
    
        for each in country_list:

            df_plot=df_input_large[df_input_large['country']==each]
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
            df_plot=df_plot['confirmed'][df_plot['confirmed']>35].reset_index(drop=True)


            ydata=df_plot
            xdata= np.arange(len(df_plot))

            traces.append(dict(
                                    x=xdata,
                                    y=ydata,
                                    type='bar',
                                    opacity=0.9,
                                    visible=True,
                                    name=each+'_Confirmed'))

            pd_beta=np.concatenate((np.array(t_init*[bmax]),
                                    np.linspace(bmax,bmin,t_intro),
                                    np.array(t_hold*[bmin]),
                                    np.linspace(bmin,bmax,t_relax),))

            I0=df_plot[0]    
            N0=np.array(df_plot)[-1]/0.04
            S0=N0-I0
            R0=0


            SIR=np.array([S0,I0,R0])


            propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                                'infected':I0,
                                                'recoverd':R0})

            for each_beta in pd_beta:

                new_delta_vec=SIR_model(SIR,each_beta,gamma,N0)

                SIR=SIR+new_delta_vec

                propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                            'infected':SIR[1],
                                                            'recovered':SIR[2]}, ignore_index=True)
                


            traces.append(dict(
                                    x=propagation_rates.index,
                                    y=propagation_rates.infected,
                                    mode='markers+lines',
                                    legend_title="Legend Title",
                                    opacity=0.9,
                                    visible=True,
                                    name=each+'_Infected'))


    return {
                        'data': traces,
                        'layout': dict (width=1280,
                                        height=720,
                                        title= 'Scenario SIR simulations  (demonstration purposes only)',
                                        xaxis={ 'title':'Time in days',
                                         'tickfont':dict(size=14,color="#7f7f7f")},
                                        yaxis=my_yaxis
                        'shapes'=dict(type="rect",
                                      xref="x",yref="y",
                                      x0=min(x0),y0=min(y0),
                                      x1=max(x0),y1=max(y0),
                                      opacity=0.2,fillcolor="blue",
                                      line_color="blue",)
        }
    
    

In [ ]:
 app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
0.1
log
{'title': 'Confirmed infected people (Source: John Hopkins,linear-scale)', 'type': 'linear', 'tickfont': {'size': 14, 'color': '#7f7f7f'}}
    susceptible   infected  recoverd  recovered
0  1.550129e+08  67.599993       NaN        5.2
    susceptible   infected  recoverd  recovered
0  1.550129e+08  67.599993       NaN   5.200000
1  1.550129e+08  87.879978       NaN  11.959999
    susceptible    infected  recoverd  recovered
0  1.550129e+08   67.599993       NaN   5.200000
1  1.550129e+08   87.879978       NaN  11.959999
2  1.550128e+08  114.243949       NaN  20.747997
    susceptible    infected  recoverd  recovered
0  1.550129e+08   67.599993       NaN   5.200000
1  1.550129e+08   87.879978       NaN  11.959999
2  1.550128e+08  114.2439

     susceptible     infected  recoverd    recovered
0   1.550129e+08    67.599993       NaN     5.200000
1   1.550129e+08    87.879978       NaN    11.959999
2   1.550128e+08   114.243949       NaN    20.747997
3   1.550128e+08   148.517094       NaN    32.172392
4   1.550127e+08   193.072153       NaN    47.024101
5   1.550126e+08   250.993679       NaN    66.331317
6   1.550125e+08   326.291577       NaN    91.430685
7   1.550124e+08   424.178699       NaN   124.059842
8   1.550122e+08   551.431709       NaN   166.477712
9   1.550120e+08   716.860200       NaN   221.620883
10  1.550117e+08   931.916523       NaN   293.306903
11  1.550114e+08  1211.488534       NaN   386.498555
12  1.550109e+08  1574.930099       NaN   507.647409
13  1.550102e+08  2047.400665       NaN   665.140419
14  1.550094e+08  2661.606533       NaN   869.880485
15  1.550084e+08  3460.064239       NaN  1136.041139
16  1.550070e+08  4498.042474       NaN  1482.047562
17  1.550052e+08  5847.385807       NaN  1931.

     susceptible       infected  recoverd      recovered
0   1.550129e+08      67.599993       NaN       5.200000
1   1.550129e+08      87.879978       NaN      11.959999
2   1.550128e+08     114.243949       NaN      20.747997
3   1.550128e+08     148.517094       NaN      32.172392
4   1.550127e+08     193.072153       NaN      47.024101
5   1.550126e+08     250.993679       NaN      66.331317
6   1.550125e+08     326.291577       NaN      91.430685
7   1.550124e+08     424.178699       NaN     124.059842
8   1.550122e+08     551.431709       NaN     166.477712
9   1.550120e+08     716.860200       NaN     221.620883
10  1.550117e+08     931.916523       NaN     293.306903
11  1.550114e+08    1211.488534       NaN     386.498555
12  1.550109e+08    1574.930099       NaN     507.647409
13  1.550102e+08    2047.400665       NaN     665.140419
14  1.550094e+08    2661.606533       NaN     869.880485
15  1.550084e+08    3460.064239       NaN    1136.041139
16  1.550070e+08    4498.042474

0.1
linear
0.1
log
{'title': 'Confirmed infected people (Source: John Hopkins,linear-scale)', 'type': 'linear', 'tickfont': {'size': 14, 'color': '#7f7f7f'}}
    susceptible   infected  recoverd  recovered
0  1.550129e+08  67.599993       NaN        5.2
    susceptible   infected  recoverd  recovered
0  1.550129e+08  67.599993       NaN   5.200000
1  1.550129e+08  87.879978       NaN  11.959999
    susceptible    infected  recoverd  recovered
0  1.550129e+08   67.599993       NaN   5.200000
1  1.550129e+08   87.879978       NaN  11.959999
2  1.550128e+08  114.243949       NaN  20.747997
    susceptible    infected  recoverd  recovered
0  1.550129e+08   67.599993       NaN   5.200000
1  1.550129e+08   87.879978       NaN  11.959999
2  1.550128e+08  114.243949       NaN  20.747997
3  1.550128e+08  148.517094       NaN  32.172392
    susceptible    infected  recoverd  recovered
0  1.550129e+08   67.599993       NaN   5.200000
1  1.550129e+08   87.879978       NaN  11.959999
2  1.550128e+0

49  1.546800e+08  107034.977306       NaN  225927.785237
     susceptible       infected  recoverd      recovered
0   1.550129e+08      67.599993       NaN       5.200000
1   1.550129e+08      87.879978       NaN      11.959999
2   1.550128e+08     114.243949       NaN      20.747997
3   1.550128e+08     148.517094       NaN      32.172392
4   1.550127e+08     193.072153       NaN      47.024101
5   1.550126e+08     250.993679       NaN      66.331317
6   1.550125e+08     326.291577       NaN      91.430685
7   1.550124e+08     424.178699       NaN     124.059842
8   1.550122e+08     551.431709       NaN     166.477712
9   1.550120e+08     716.860200       NaN     221.620883
10  1.550117e+08     931.916523       NaN     293.306903
11  1.550114e+08    1211.488534       NaN     386.498555
12  1.550109e+08    1574.930099       NaN     507.647409
13  1.550102e+08    2047.400665       NaN     665.140419
14  1.550094e+08    2661.606533       NaN     869.880485
15  1.550084e+08    3460.064239